In [1]:
!git clone https://github.com/nqh-tq-32052503/Moal.git

Cloning into 'Moal'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 253 (delta 52), reused 61 (delta 37), pack-reused 171 (from 1)
Receiving objects: 100% (253/253), 500.58 KiB | 13.53 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [2]:
%cd /kaggle/working/Moal/Moal

import os
os.makedirs("/root/autodl-tmp/data/cifar224")

/kaggle/working/Moal/Moal


In [3]:
import json

with open("exps/Moal_cifar224_bilora.json", "r") as f:
    args = json.load(f)
print(type(args))
args['tuned_epoch'] = 20
args['progreesive_epoch'] = 15
args['batch_size'] = 64

<class 'dict'>


In [4]:
from utils.data_manager import DataManager
from utils import factory
from utils.toolkit import count_parameters

In [5]:
data_manager = DataManager(
        args["dataset"],
        args["shuffle"],
        args["seed"],
        args["init_cls"],
        args["increment"],
        args,
        "/root/autodl-tmp/data/cifar224"
    )

100%|██████████| 169M/169M [00:03<00:00, 48.4MB/s]


[43, 92, 14, 57, 89, 20, 34, 69, 21, 3, 77, 33, 54, 86, 49, 79, 98, 15, 16, 61, 46, 59, 39, 12, 56, 50, 27, 87, 45, 78, 24, 0, 7, 51, 4, 73, 8, 35, 10, 13, 70, 74, 31, 55, 82, 5, 67, 75, 22, 76, 47, 83, 90, 1, 6, 84, 94, 66, 88, 91, 23, 29, 64, 37, 72, 18, 11, 44, 40, 93, 80, 30, 81, 68, 32, 58, 63, 2, 17, 65, 38, 41, 97, 48, 25, 28, 52, 99, 9, 36, 85, 62, 60, 96, 95, 53, 19, 42, 26, 71]


In [6]:
args["nb_classes"] = data_manager.nb_classes # update args
args["nb_tasks"] = data_manager.nb_tasks
args["model_name"] = "adapt_ac_com_sdc_ema_auto"

In [7]:
model = factory.get_model(args["model_name"], args)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

This is for the BaseNet initialization.


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

After BaseNet initialization.


In [8]:
import logging
import time
cnn_curve, nme_curve = {"top1": [], "top5": []}, {"top1": [], "top5": []}

all_his_acc = []
for task in range(data_manager.nb_tasks):
    st = time.time()
    logging.info("All params: {}".format(count_parameters(model._network)))
    logging.info(
        "Trainable params: {}".format(count_parameters(model._network, True))
    )
    model.incremental_train(data_manager)
    cnn_accy, nme_accy = model.eval_task(model._cur_task)
    model.after_task()
    en = time.time()
    print("[TIME TIME TIME] Task=", task, "Elapsed time: ", en - st)
    if nme_accy is not None:
        logging.info("CNN: {}".format(cnn_accy["grouped"]))
        logging.info("NME: {}".format(nme_accy["grouped"]))

        all_his_acc.append(cnn_accy["grouped"].values())
        cnn_curve["top1"].append(cnn_accy["top1"])
        cnn_curve["top5"].append(cnn_accy["top5"])

        nme_curve["top1"].append(nme_accy["top1"])
        nme_curve["top5"].append(nme_accy["top5"])

        logging.info("CNN top1 curve: {}".format(cnn_curve["top1"]))
        logging.info("CNN top5 curve: {}".format(cnn_curve["top5"]))
        logging.info("NME top1 curve: {}".format(nme_curve["top1"]))
        logging.info("NME top5 curve: {}\n".format(nme_curve["top5"]))

        print('Average Accuracy (CNN):', sum(cnn_curve["top1"])/len(cnn_curve["top1"]))
        print('Average Accuracy (NME):', sum(nme_curve["top1"])/len(nme_curve["top1"]))

        logging.info("Average Accuracy (CNN): {}".format(sum(cnn_curve["top1"])/len(cnn_curve["top1"])))
        logging.info("Average Accuracy (NME): {}".format(sum(nme_curve["top1"])/len(nme_curve["top1"])))
    else:
        logging.info("No NME accuracy.")
        logging.info("CNN: {}".format(cnn_accy["grouped"]))

        cnn_accy["grouped"].pop('total')
        cnn_accy["grouped"].pop('old')
        cnn_accy["grouped"].pop('new')
        all_his_acc.append(list(cnn_accy["grouped"].values()))

        cnn_curve["top1"].append(cnn_accy["top1"])
        cnn_curve["top5"].append(cnn_accy["top5"])

        logging.info("CNN top1 curve: {}".format(cnn_curve["top1"]))
        logging.info("CNN top5 curve: {}\n".format(cnn_curve["top5"]))

        print('Average Accuracy (CNN):', sum(cnn_curve["top1"])/len(cnn_curve["top1"]))
        logging.info("Average Accuracy (CNN): {} \n".format(sum(cnn_curve["top1"])/len(cnn_curve["top1"])))
logging.info("All History Accuracy (CNN): {} \n".format(all_his_acc))

Learning on 0-10
Train dataset size: 5000


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


114,877,825 total parameters.
29,079,169 training parameters.
backbone.cls_token_grow 3840000
backbone.pos_embed_grow 918528
backbone.blocks.0.attn.lora_A_k.0.weight 49152
backbone.blocks.0.attn.lora_A_k.1.weight 49152
backbone.blocks.0.attn.lora_A_k.2.weight 49152
backbone.blocks.0.attn.lora_A_k.3.weight 49152
backbone.blocks.0.attn.lora_A_k.4.weight 49152
backbone.blocks.0.attn.lora_A_k.5.weight 49152
backbone.blocks.0.attn.lora_A_k.6.weight 49152
backbone.blocks.0.attn.lora_A_k.7.weight 49152
backbone.blocks.0.attn.lora_A_k.8.weight 49152
backbone.blocks.0.attn.lora_A_k.9.weight 49152
backbone.blocks.0.attn.lora_B_k.0.weight 49152
backbone.blocks.0.attn.lora_B_k.1.weight 49152
backbone.blocks.0.attn.lora_B_k.2.weight 49152
backbone.blocks.0.attn.lora_B_k.3.weight 49152
backbone.blocks.0.attn.lora_B_k.4.weight 49152
backbone.blocks.0.attn.lora_B_k.5.weight 49152
backbone.blocks.0.attn.lora_B_k.6.weight 49152
backbone.blocks.0.attn.lora_B_k.7.weight 49152
backbone.blocks.0.attn.lora_B

100%|██████████| 79/79 [01:16<00:00,  1.03it/s]


Task 0, Epoch 1/20 => Loss 2.269, Train_accy 25.42, Test_accy 46.70


...

100%|██████████| 50/50 [00:09<00:00,  5.44it/s]


best_loss: 0.12942341649532318
Computing class relations...
Old means shape: (90, 768)
New means shape: (10, 768)
Class relations: [94 91 94 95 94 92 94 94 94 94 92 92 92 93 94 92 92 92 93 92 92 92 94 92
 92 92 92 94 94 92 94 92 94 92 94 94 94 94 94 94 93 93 93 93 93 93 94 93
 93 93 94 94 94 94 93 94 94 94 93 94 99 90 90 90 90 93 90 98 90 99 98 94
 90 99 98 90 98 98 90 98 98 90 98 90 98 90 93 90 98 98]
Building feature dataset...
Extract prototypes for known classes...
Total feature dataset size: 50000
Feature dataset dimension: 768
Label dataset size: 50000
Label dataset classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Incremental class alignment (Knowledge Memorization)...
Knowledge Memorization completed.
Updated weight m

Alignment: 100%|██████████| 782/782 [00:28<00:00, 27.09batch/s]

numpy inverse
Finish one task 


[TIME TIME TIME] Task= 9 Elapsed time:  3297.454295396805
Average Accuracy (CNN): 17.946999999999996


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Dữ liệu Giả định
# List 10 giá trị số thực (từ 0 đến 100)
task_values = [81.4, 55.2, 42.03, 34.89, 30.11, 26.56, 23.62, 21.26, 19.39, 17.94]
task_labels = [f"Task {i+1}" for i in range(len(task_values))]
data_range = len(task_labels)

# 2. Thiết lập Phong cách Đồ thị
# Sử dụng phong cách của Seaborn để đồ thị đẹp mắt hơn
sns.set_theme(style="whitegrid") 

# 3. Tạo Hình vẽ và Trục chính (Cho biểu đồ Cột)
fig, ax1 = plt.subplots(figsize=(10, 6))

# --- BIỂU ĐỒ CỘT (Bar Chart) ---
bar_colors = sns.color_palette("viridis", data_range) # Chọn bảng màu
bars = ax1.bar(task_labels, task_values, color=bar_colors, alpha=0.7, label='Accuracy')

# Cài đặt Trục Y chính
ax1.set_xlabel('Tasks', fontsize=12)
ax1.set_ylabel('Accuracy (%)', color='blue', fontsize=12)
ax1.tick_params(axis='y', labelcolor='blue')
ax1.set_ylim(0, 105) # Giới hạn trục Y từ 0 đến 105 để biểu đồ rõ ràng hơn
ax1.grid(axis='y', linestyle='--', alpha=0.6) # Lưới ngang

# Tùy chỉnh các nhãn trục X
ax1.set_xticks(range(data_range))
ax1.set_xticklabels(task_labels, rotation=45, ha='right') # Xoay nhãn để dễ đọc

# Hiển thị giá trị trên đỉnh mỗi cột
for bar in bars:
    height = bar.get_height()
    ax1.annotate(f'{height:.1f}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),  # Dịch chuyển 3 points theo trục Y
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=9)


# 4. Tạo Trục kép (Cho biểu đồ Đường)
# Tạo trục y thứ hai chia sẻ cùng trục x (Twin axis)
ax2 = ax1.twinx() 

# --- BIỂU ĐỒ ĐƯỜNG (Line Chart) ---
line_color = 'red'
# ax2.plot() để vẽ biểu đồ đường
ax2.plot(task_labels, task_values, color=line_color, marker='o', linestyle='-', linewidth=2, label='Accuracy')

# Cài đặt Trục Y phụ
ax2.set_ylabel('Accuracy', color=line_color, fontsize=12) 
ax2.tick_params(axis='y', labelcolor=line_color)
ax2.set_ylim(0, 105) # Đồng bộ giới hạn Y với trục chính

# 5. Hoàn thiện Đồ thị
plt.title('Accuracy across tasks', fontsize=14, fontweight='bold')
fig.tight_layout() # Điều chỉnh layout để tránh cắt xén nhãn

# Tạo legend tùy chỉnh từ hai trục
# Lấy các handler và labels từ cả hai trục
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

# Hiển thị biểu đồ
plt.show()